### 1. Importer les bibliothèques
Importation de toutes les bibliothèques nécessaires pour la manipulation des données, la visualisation, le prétraitement, la modélisation et l'évaluation.

In [1]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import os
import glob
from sklearn.impute import SimpleImputer
import numpy as np 


### 2. Charger les données
Chargement du jeu de données depuis le fichier CSV spécifié.
*Note : Assurez-vous que le chemin `\\France_Air_Quality\\DataExtract.csv` est correct par rapport à l'emplacement de votre notebook ou fournissez un chemin absolu. Utiliser des chaînes brutes `r'...'` ou des barres obliques `/` peut parfois être plus fiable sur différents systèmes d'exploitation.*

In [2]:
data_directory = r"C:\Projet_ESME\Pollution\Pollution\France_Air_Quality"

print(f"Chargement des Données depuis : {data_directory}")
df = None 

csvs = os.path.join(data_directory, "*.csv")
csv_files = glob.glob(csvs)
df_list = []
for file in csv_files:
    temp_df = pd.read_csv(file, low_memory=False)
    df_list.append(temp_df)
        
    if df_list:
        df = pd.concat(df_list, ignore_index=True)

print(f"Chargement des Données terminé : {len(df_list)} fichiers concaténés")
print(f"Nombre de lignes : {df.shape[0]}")

Chargement des Données depuis : C:\Projet_ESME\Pollution\Pollution\France_Air_Quality
Chargement des Données terminé : 42 fichiers concaténés
Nombre de lignes : 584379
Chargement des Données terminé : 42 fichiers concaténés
Nombre de lignes : 584379


### 3. Exploration Initiale et Visualisation des Manquants
Affichage des premières lignes, des informations générales et visualisation des valeurs manquantes si les données ont été chargées.

In [ ]:
def exploration_initiale(df):
    if df is not None:
        print("\n Exploration Initiale ")
        print("Aperçu des 5 premières lignes :")
        display(df.head())

        print("\nInformations sur le DataFrame :")
        display(df.info())

        print("\n Visualisation des Valeurs Manquantes ")
        print("\nVisualisation (Matrice)...")
        try:
            msno.matrix(df)
            plt.show()
        except Exception as e:
            print(f"Erreur lors de la création de la matrice missingno : {e}")

        print("\nVisualisation (Heatmap)...")
        try:
            msno.heatmap(df)
            plt.show()
        except Exception as e:
            print(f"Erreur lors de la création de la heatmap missingno : {e}")
    else:
        print("DataFrame non chargé, exploration impossible.")

exploration_initiale(df)

In [8]:
def analyze_pollutant_distribution(df, pollutant_col='Air Pollutant', print_results=True):
    if df is not None and pollutant_col in df.columns:
        # 1. Compter les occurrences des polluants
        pollutant_counts = df[pollutant_col].value_counts(dropna=True)
        
        # 2. Transformer la Série en DataFrame et garder les 20 premiers en termes de fréquence
        pollutant_table = pollutant_counts.reset_index().head(20)
        pollutant_table.columns = [pollutant_col, 'Frequency']
        
        # 3. Trier le DataFrame par fréquence (ordre décroissant)
        pollutant_table_sorted = pollutant_table.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

        # Extraire les 20 polluants les plus fréquents en liste
        top_pollutants_list = pollutant_table_sorted[pollutant_col].tolist()

        # Affichage optionnel des résultats
        if print_results:
            print(f"----- Distribution des Polluants dans la colonne '{pollutant_col}' -----")
            try:
                display(pollutant_table_sorted)
            except NameError:
                print(pollutant_table_sorted.to_string())  # .to_string() pour afficher tout sans troncature
            print(f"\nNombre total de types de polluants uniques trouvés : {len(pollutant_counts)}")
            print(f"\nListe des 20 polluants les plus fréquents : {top_pollutants_list}")

        return pollutant_table_sorted, top_pollutants_list
    else:
        print(f"Erreur : Le DataFrame est vide ou la colonne '{pollutant_col}' est absente.")
        return None, None

# Appel de la fonction pour analyser la distribution des polluants
pollutant_table, top_pollutants = analyze_pollutant_distribution(df)

----- Distribution des Polluants dans la colonne 'Air Pollutant' -----


,Air Pollutant,Frequency
0,O3,187777
1,NO2,130374
2,PM10,90120
3,SO2,80948
4,NO,27170
5,PM2.5,24318
6,CO,11802
7,NOX as NO2,11639
8,NOX,5248
9,C6H6,3271



Nombre total de types de polluants uniques trouvés : 102

Liste des 20 polluants les plus fréquents : ['O3', 'NO2', 'PM10', 'SO2', 'NO', 'PM2.5', 'CO', 'NOX as NO2', 'NOX', 'C6H6', 'SPM', 'BaP in PM10', 'SA', 'Pb in PM10', 'As in PM10', 'Cd in PM10', 'Ni in PM10', 'C6H5-CH3', 'C6H5-C2H5', 'o-C6H4-(CH3)2']


Pour la suite, nous allons sélectionner les polluants suivants et générer un modèle spécifique pour chacun : ['O3', 'NO2', 'PM10', 'SO2', 'NO', 'PM2.5', 'CO', 'NOX'].

### 4. Nettoyage des Données
Suppression des colonnes jugées inutiles et des lignes où la variable cible (`Air Pollution Level`) est manquante ou non numérique.

In [4]:
if df is not None:
    print("\n--- Nettoyage des Données ---")
    # 4.1 Suppression des Colonnes
    initial_cols = df.shape[1]
    drop_cols = ['Country', 'Air Quality Network', 'Air Quality Network Name',
                 'Air Quality Station EoI Code', 'Air Quality Station Name',
                 'Sampling Point Id','Air Pollutant Descritpion', 'Data Aggregation Process Id',
                 'Data Aggregation Process', 'Unit Of Air Pollution Level',
                 'Verification', 'City Code', 'Source Of Data Flow',
                 'Calculation Time', 'Link to raw data (only E1a/validated data from AQ e-Reporting)',
                 'Observation Frequency']

    cols_to_drop_existing = [col for col in drop_cols if col in df.columns]
    if cols_to_drop_existing:
        df.drop(columns=cols_to_drop_existing, inplace=True)
        print(f"Colonnes supprimées : {len(cols_to_drop_existing)}")
    else:
        print("Aucune des colonnes spécifiées pour suppression n'a été trouvée.")
    print(f"Dimensions après suppression colonnes : {df.shape}")

    # 4.2 Traitement de la Cible et Suppression des Lignes Manquantes/Invalides
    if 'Air Pollution Level' in df.columns:
        initial_rows = df.shape[0]
        df['Air Pollution Level'] = pd.to_numeric(df['Air Pollution Level'], errors='coerce')
        rows_before_nan_drop = df.shape[0]
        df.dropna(subset=['Air Pollution Level'], inplace=True)
        rows_after_nan_drop = df.shape[0]
        print(f"Lignes supprimées (cible manquante ou non numérique) : {initial_rows - rows_after_nan_drop}")
        if rows_before_nan_drop > rows_after_nan_drop :
            print(f" -> Dont {rows_before_nan_drop - rows_after_nan_drop} lignes avec cible non-numérique convertie en NaN.")
        print(f"Dimensions après suppression lignes : {df.shape}")
    else:
        print("AVERTISSEMENT : Colonne cible 'Air Pollution Level' non trouvée.")

    # Vérifier si le DataFrame est vide après nettoyage
    if df.empty:
        print("\n *** ATTENTION : Le DataFrame est vide après nettoyage. Les étapes suivantes échoueront. ***")
        df = None # Marquer df comme invalide
else:
    print("DataFrame non chargé, nettoyage impossible.")


--- Nettoyage des Données ---
Colonnes supprimées : 15
Dimensions après suppression colonnes : (584379, 12)
Lignes supprimées (cible manquante ou non numérique) : 1313
 -> Dont 1313 lignes avec cible non-numérique convertie en NaN.
Dimensions après suppression lignes : (583066, 12)


### 5. Séparation Features / Cible et Identification des Types
Séparation des variables explicatives (X) et de la cible (y), puis identification des colonnes numériques et catégorielles à utiliser.

In [10]:
X, y, num_features, cat_features = None, None, [], [] # Initialisation

if df is not None:
    print("\n--- Séparation Features / Cible et Identification Types ---")
    if 'Air Pollution Level' in df.columns:
        X = df.drop(columns=['Air Pollution Level'])
        y = df['Air Pollution Level']
        print(f"Dimensions Features (X) : {X.shape}")
        print(f"Dimensions Cible (y) : {y.shape}")

        num_features_potential = ['Year', 'Longitude', 'Latitude', 'Altitude', 'City Population']
        cat_features_potential = ['Air Pollutant', 'Air Quality Station Type', 'Air Quality Station Area', 'City']

        num_features = [col for col in num_features_potential if col in X.columns]
        cat_features = [col for col in cat_features_potential if col in X.columns]

        print(f"\nFeatures numériques identifiées : {num_features}")
        print(f"Features catégorielles identifiées : {cat_features}")

        # Vérification des colonnes non traitées
        all_features_in_transformer = set(num_features + cat_features)
        all_columns_in_X = set(X.columns)
        unhandled_columns = all_columns_in_X - all_features_in_transformer
        if unhandled_columns:
            print(f"\nColonnes non explicitement traitées (seront affectées par 'remainder') : {unhandled_columns}")
        else:
            print("\nToutes les colonnes de X sont prises en compte par les features numériques ou catégorielles.")

    else:
        print("Colonne cible 'Air Pollution Level' non trouvée dans le DataFrame nettoyé.")
else:
    print("DataFrame non disponible pour la séparation Features/Cible.")

### 6. Définition du Pipeline de Prétraitement
Création d'un `ColumnTransformer` pour appliquer l'imputation et la mise à l'échelle/encodage aux différents types de features.

In [12]:
preprocessor = None # Initialisation

if X is not None and (num_features or cat_features):
    print("\n--- Définition du Pipeline de Prétraitement ---")
    transformers_list = []

    if num_features:
        num_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])
        transformers_list.append(('num', num_pipeline, num_features))
        print("Pipeline numérique défini (Imputer + Scaler).")

    if cat_features:
        cat_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])
        transformers_list.append(('cat', cat_pipeline, cat_features))
        print("Pipeline catégoriel défini (Imputer + OneHotEncoder).")

    # 'remainder=drop' pour supprimer les colonnes non listées explicitement
    preprocessor = ColumnTransformer(
        transformers=transformers_list,
        remainder='drop'
    )
    print(f"Pipeline de prétraitement final défini (remainder='{preprocessor.remainder}').")
else:
    print("Impossible de définir le pipeline de prétraitement (X manquant ou aucune feature identifiée).")

### 7. Séparation Train / Test (Division Temporelle)
Division des données en ensembles d'entraînement et de test en respectant l'ordre chronologique basé sur l'année (`Year`) pour une évaluation réaliste. Utilisation de la méthode proportionnelle sur données triées.

In [13]:
X_train, X_test, y_train, y_test = None, None, None, None # Initialisation

if df is not None and X is not None and y is not None : # S'assurer que df, X et y existent
    print("\n--- Séparation Train / Test (Division Temporelle) ---")

    # 1. Trier le DataFrame COMPLET (avant de séparer X et y !) par année
    # Utiliser le df nettoyé qui contient encore X et y
    df_sorted = df.sort_values(by='Year').reset_index(drop=True)
    print(f"Données triées par année. Année min: {df_sorted['Year'].min()}, Année max: {df_sorted['Year'].max()}")

    # 2. Définir la proportion pour le test set
    test_proportion = 0.20 # 20% des données les plus récentes pour le test
    split_index = int(len(df_sorted) * (1 - test_proportion))
    print(f"Index de coupure pour {test_proportion*100:.0f}% de test : {split_index} (sur {len(df_sorted)} lignes)")

    # 3. Créer les DataFrames train/test à partir du DataFrame trié
    df_train = df_sorted.iloc[:split_index].copy()
    df_test = df_sorted.iloc[split_index:].copy()

    # 4. Séparer X et y DANS chaque ensemble (train et test)
    if not df_train.empty and 'Air Pollution Level' in df_train.columns:
        X_train = df_train.drop(columns=['Air Pollution Level'])
        y_train = df_train['Air Pollution Level']
    else:
         print("ERREUR: df_train est vide ou manque la colonne cible.")

    if not df_test.empty and 'Air Pollution Level' in df_test.columns:
        X_test = df_test.drop(columns=['Air Pollution Level'])
        y_test = df_test['Air Pollution Level']
    else:
         print("ERREUR: df_test est vide ou manque la colonne cible.")


    # 5. Vérifier les tailles et la couverture temporelle
    if X_train is not None and X_test is not None:
        print(f"\nTaille Train Set (X/y): {X_train.shape[0]}")
        print(f"Années couvertes par Train: {df_train['Year'].min()} - {df_train['Year'].max()}")
        print(f"Taille Test Set (X/y): {X_test.shape[0]}")
        print(f"Années couvertes par Test: {df_test['Year'].min()} - {df_test['Year'].max()}")
    else:
        print("Problème lors de la création de X/y train ou test après la division temporelle.")

else:
    print("Impossible de faire la séparation Train/Test (df, X ou y manquants).")

### 8. Création et Entraînement du Modèle
Définition du pipeline final intégrant le prétraitement et le modèle `RandomForestRegressor`, puis entraînement sur l'ensemble d'entraînement.

In [14]:
model = None # Initialisation

if preprocessor is not None and X_train is not None and y_train is not None:
    print("\n--- Création et Entraînement du Modèle ---")

    # Création du pipeline final Modèle
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
    ])

    print("Entraînement du modèle RandomForestRegressor...")
    try:
        # Vérification finale du type de y_train
        if not pd.api.types.is_numeric_dtype(y_train):
             print("ERREUR critique : y_train n'est pas numérique avant l'entraînement !")
        else:
            model.fit(X_train, y_train)
            print("Entraînement terminé.")
    except Exception as e:
        print(f"Une erreur est survenue pendant l'entraînement : {e}")
        model = None # Invalider le modèle

else:
    print("Impossible de créer ou entraîner le modèle (prérequis manquants).")

### 9. Prédiction sur l'Ensemble de Test
Utilisation du modèle entraîné pour faire des prédictions sur l'ensemble de test.

In [15]:
y_pred = None # Initialisation

if model is not None and X_test is not None:
    print("\n--- Prédiction sur l'Ensemble de Test ---")
    try:
        y_pred = model.predict(X_test)
        print("Prédictions réalisées.")
        print(f"Exemple de 5 premières prédictions : {y_pred[:5]}")
    except Exception as e:
        print(f"Une erreur est survenue pendant la prédiction : {e}")
        y_pred = None
else:
    print("Impossible de faire des prédictions (modèle non entraîné ou X_test manquant).")

### 10. Évaluation du Modèle
Calcul des métriques MSE et RMSE pour évaluer la performance du modèle sur l'ensemble de test.

In [16]:
if y_test is not None and y_pred is not None:
    print("\n--- Évaluation du Modèle ---")
    try:
        # Vérification finale du type de y_test
        if not pd.api.types.is_numeric_dtype(y_test):
            print("AVERTISSEMENT: y_test n'est pas numérique, tentative de nettoyage...")
            y_test_numeric = pd.to_numeric(y_test, errors='coerce')
            valid_indices_test = ~y_test_numeric.isna()
            if not valid_indices_test.all():
                 initial_len = len(y_test_numeric)
                 y_test_cleaned = y_test_numeric[valid_indices_test]
                 y_pred_cleaned = y_pred[valid_indices_test] # Filtrer y_pred aussi!
                 print(f"Suppression de {initial_len - len(y_test_cleaned)} lignes où y_test n'était pas numérique.")
                 if len(y_test_cleaned) != len(y_pred_cleaned):
                      print("ERREUR: Mismatch de longueur après nettoyage y_test/y_pred.")
                      raise ValueError("Incohérence des longueurs après nettoyage de y_test.")
                 y_test_eval = y_test_cleaned
                 y_pred_eval = y_pred_cleaned
            else:
                 y_test_eval = y_test_numeric
                 y_pred_eval = y_pred # Pas besoin de filtrer y_pred si y_test était ok

        else:
             y_test_eval = y_test
             y_pred_eval = y_pred

        if y_test_eval.empty:
             print("ERREUR: y_test est vide après nettoyage. Impossible d'évaluer.")
        else:
            mse = mean_squared_error(y_test_eval, y_pred_eval)
            rmse = np.sqrt(mse) # Calcul manuel de RMSE
            print(f"Erreur Quadratique Moyenne (MSE) : {mse:.4f}")
            print(f"Racine de l'Erreur Quadratique Moyenne (RMSE) : {rmse:.4f}")

            # Affichage du graphique Prédit vs Réel
            print("\nAffichage du graphique Prédit vs Réel...")
            plt.figure(figsize=(8, 8))
            sns.scatterplot(x=y_test_eval, y=y_pred_eval, alpha=0.5)
            # Déterminer les limites pour la ligne diagonale
            min_val = min(y_test_eval.min(), y_pred_eval.min())
            max_val = max(y_test_eval.max(), y_pred_eval.max())
            plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--') # Ligne y=x
            plt.xlabel("Vraies Valeurs (y_test)")
            plt.ylabel("Valeurs Prédites (y_pred)")
            plt.title("Comparaison Valeurs Prédites vs Réelles (Test Set)")
            plt.grid(True)
            # Limiter les axes pour mieux voir la dispersion si les valeurs sont très grandes
            # plt.xlim(min_val, max_val)
            # plt.ylim(min_val, max_val)
            plt.show()


    except Exception as e:
        print(f"Une erreur est survenue lors de l'évaluation : {e}")
else:
    print("Impossible d'évaluer le modèle (y_test ou y_pred manquants/invalides).")

print("\n--- Fin du Notebook ---")